## Autoregressive Models

In [3]:
import warnings

import matplotlib.pyplot as plt 
import pandas as pd
import plotly.express as px 
from pymongo import MongoClient
from sklearn.metrics import  median_absolute_error
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.ar_model import AutoReg

warnings.simplefilter(action="ignore", category=FutureWarning)

#### Prepare Data

Import

In [ ]:
client = MongoClient(host="localhost", port=27017)
db = client["air-quality"]
nairobi = db["nairobi"]

The Wrangle function to import data:

In [ ]:
def wrangle(collection):
    results = collection.find(
        {"metadata.site": 29, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )

    # Read data into DataFrame
    df = pd.DataFrame(list(results)).set_index("timestamp")

    # Localize timezone
    df.index = df.index.tz_localize("UTC").tz_convert("Africa/Nairobi")

    # Remove outliers
    df = df[df["P2"] < 500]

    # Resample to 1hr window
    y = df["P2"].resample("1H").mean().fillna(method='ffill')

    return y

In [7]:
y = pd.read_csv("data/nairobi-city-air-quality.csv").set_index("timestamp")[y["P2"] < 500]
y.head()

,P2
timestamp,
00:02.5,34.43
05:03.9,30.53
10:04.4,22.80
15:04.2,13.30
20:04.9,16.57
